<a href="https://colab.research.google.com/github/chrisseub/ML-Practice-Rep/blob/main/EGG_SIZE_PREDICTION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

IMPORT THE LIBRARIES

In [48]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

  IMPORT THE EGG QUALITY DATA

In [9]:
Egg_Traits = pd.read_excel('/content/main.xls')

#VIEWING THE DATA
print(Egg_Traits.head()) # Print the first few rows
Egg_Traits.shape


  STRAIN          AGE  EGG_WEIGHT   EGG_LENGTH  ...  Shell surface area Yolk Index Shape Index Outcome
0  BROWN  EARLY LAYER         52.5        5.75  ...           63.770823  43.902439   69.565217   Small
1  BROWN  EARLY LAYER         62.6        5.70  ...           71.623566  47.619048   75.438596   Large
2  BROWN  EARLY LAYER         61.0        5.50  ...           70.410037  45.000000   78.181818   Large
3  BROWN  EARLY LAYER         64.1        5.70  ...           72.751706  47.500000   77.192982   Large
4  BROWN  EARLY LAYER         53.9        5.60  ...           64.888161  55.000000   71.428571   Small

[5 rows x 16 columns]


(300, 16)

ANALYSING THE DATA

In [11]:
Egg_Traits.describe() #For some descriptive statistics



,EGG_WEIGHT,EGG_LENGTH,EGG_WIDTH,YOLK _ALB_WEIGHT,SHELL_WEIGTH,SHELL_THICKNESS,Haugh Unit,Shell surface area,Yolk Index,Shape Index
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,299.000000,300.000000,296.000000,300.000000
mean,63.848667,5.800800,4.340500,55.607333,7.593667,0.323033,84.539822,72.483645,48.002424,75.555584
std,6.304399,0.306651,0.291033,5.840976,0.704686,0.042691,0.744175,4.739732,4.481038,5.981744
min,49.700000,5.100000,0.300000,43.000000,5.900000,0.200000,81.801110,61.505237,21.621622,5.263158
25%,59.200000,5.587500,4.200000,51.300000,7.100000,0.290000,84.057052,69.031801,45.238095,73.098388
50%,63.650000,5.800000,4.300000,55.500000,7.500000,0.330000,84.548793,72.414210,47.619048,75.545385
75%,68.600000,6.000000,4.500000,60.350000,8.025000,0.350000,85.045960,76.083555,50.166667,78.181818
max,78.900000,7.000000,4.900000,70.200000,9.900000,0.420000,86.421165,83.442576,61.764706,89.090909


In [14]:
Egg_Traits['Outcome'].value_counts()



,count
Outcome,
Large,229
Small,71


In [32]:
numeric_columns = Egg_Traits.select_dtypes(include=np.number).columns

# Calculate the mean for only numeric columns
Egg_Traits.groupby('Outcome')[numeric_columns].mean()

,EGG_WEIGHT,EGG_LENGTH,EGG_WIDTH,YOLK _ALB_WEIGHT,SHELL_WEIGTH,SHELL_THICKNESS,Haugh Unit,Shell surface area,Yolk Index,Shape Index
Outcome,,,,,,,,,,
Large,66.427948,5.890349,4.392140,57.916594,7.759825,0.317031,84.428646,74.443886,47.850931,74.856325
Small,55.529577,5.511972,4.173944,48.159155,7.057746,0.342394,84.896838,66.161177,48.510369,77.810941


In [36]:
Outcome_mapping = {'Small': 0, 'Large': 1}
# Apply the mapping to the 'Outcome' column
Egg_Traits['Outcome_Numeric'] = Egg_Traits['Outcome'].map(Outcome_mapping)
# You can now use the 'Outcome_Numeric' column for calculations
print(Egg_Traits.head())

#0 is for Small
#1 is for Large

  STRAIN          AGE  EGG_WEIGHT   EGG_LENGTH  ...  Yolk Index Shape Index Outcome Outcome_Numeric
0  BROWN  EARLY LAYER         52.5        5.75  ...   43.902439   69.565217   Small               0
1  BROWN  EARLY LAYER         62.6        5.70  ...   47.619048   75.438596   Large               1
2  BROWN  EARLY LAYER         61.0        5.50  ...   45.000000   78.181818   Large               1
3  BROWN  EARLY LAYER         64.1        5.70  ...   47.500000   77.192982   Large               1
4  BROWN  EARLY LAYER         53.9        5.60  ...   55.000000   71.428571   Small               0

[5 rows x 17 columns]


SEPERATING DATA AND LABELS

In [42]:
X = Egg_Traits.drop(columns=[Egg_Traits.columns[2], 'STRAIN', 'AGE', 'Outcome', 'Outcome_Numeric'], axis=1)
Y = Egg_Traits['Outcome_Numeric']
print(X)
print(Y)

     EGG_LENGTH  EGG_WIDTH YOLK_HEIGHT  ... Shell surface area Yolk Index  Shape Index
0          5.75       4.00         1.8  ...          63.770823  43.902439    69.565217
1          5.70       4.30           2  ...          71.623566  47.619048    75.438596
2          5.50       4.30         1.8  ...          70.410037  45.000000    78.181818
3          5.70       4.40         1.9  ...          72.751706  47.500000    77.192982
4          5.60       4.00         2.2  ...          64.888161  55.000000    71.428571
..          ...        ...         ...  ...                ...        ...          ...
295        5.30       4.15        1.85  ...          65.995674  47.435897    78.301887
296        5.65       4.25        2.05  ...          69.262486  51.898734    75.221239
297        5.70       4.30         1.9  ...          69.875722  50.000000    75.438596
298        5.60       4.20        1.75  ...          67.639242  46.052632    75.000000
299        6.10       4.20         1.8  ...

SPLITTING INTO TRAINING AND TEST DATA

In [47]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)
print(X.shape, X_train.shape, X_test.shape)

(300, 12) (240, 12) (60, 12)


In [45]:
X_train = X_train.replace('-', np.nan)
# Convert object columns to numeric, coercing errors to NaN
for col in X_train.select_dtypes(include=['object']).columns:
    X_train[col] = pd.to_numeric(X_train[col], errors='coerce')

imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)

# Create a new DataFrame with the imputed values
X_train = pd.DataFrame(X_train_imputed, columns=X_train.columns, index=X_train.index)

<ipython-input-45-886a8162595d>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_train = X_train.replace('-', np.nan)


MODEL TRAINING (LOGISTIC REGRESSION)

In [46]:
model = LogisticRegression()
model.fit(X_train, Y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()